In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)
print(f'{current_date=}')
# current_date = pd.Timestamp('2023-02-28 09:00:00')

current_date=Timestamp('2024-09-04 22:00:00')


In [2]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1007769
Connected. Call `.close()` to terminate connection gracefully.
2024-09-05 00:42:18,858 INFO: Feature view already exists, skipping creation.
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1007769
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.75s) 
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1007769
Connected. Call `.close()` to terminate connection gracefully.
2024-09-05 00:42:26,917 INFO: Feature view already exists, skipping creation.
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to projec

In [4]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1007769
Connected. Call `.close()` to terminate connection gracefully.
2024-09-05 00:43:15,926 WARNING: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.

[LightGBM] [Warning] feature_fraction is set=0.9514399636033337, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9514399636033337
[LightGBM] [Warning] bagging_fraction is set=0.45078575275817045, subsample=1.0 will be ignored. Current value: bagging_fraction=0.45078575275817045


  File "c:\Users\josec\OneDrive\Escritorio\taxi_demand_predictor\.venv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


In [5]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,28.0,2024-09-04 22:00:00
1,2,25.0,2024-09-04 22:00:00
2,3,22.0,2024-09-04 22:00:00
3,4,28.0,2024-09-04 22:00:00
4,5,24.0,2024-09-04 22:00:00
...,...,...,...
260,261,18.0,2024-09-04 22:00:00
261,262,19.0,2024-09-04 22:00:00
262,263,22.0,2024-09-04 22:00:00
263,264,21.0,2024-09-04 22:00:00


### Save these predictions in the feature store, so they can be later consumed by our Streamlit app

In [6]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1007769
Connected. Call `.close()` to terminate connection gracefully.


In [7]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Uploading Dataframe: 0.00% |          | Rows 0/265 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1007769/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1c7250a4710>, None)